# Import modules and load data

In [ ]:
import mysql.connector
import pandas as pd
import numpy as np
import pickle 

import seaborn as sns
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn import model_selection
import xgboost as xgb
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score
from sklearn.model_selection import RandomizedSearchCV
from collections import Counter
import pickle
import operator
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import make_scorer
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
import time
acc = make_scorer(accuracy_score)


from sklearn.multiclass import OneVsRestClassifier

In [ ]:
import random

def sampleData(DataFrame, ratio):
    df_size = len(DataFrame.index)
    sample_size = int(round(ratio * df_size))
    if sample_size == df_size:
        sample_size = sample_size - 1
        indexes = random.sample(range(df_size), sample_size)
        sample = DataFrame.iloc[indexes, : ]
        return sample

    elif sample_size == 0:
        sample_size = 1
        indexes = random.sample(range(df_size), sample_size)
        sample = DataFrame.iloc[indexes, : ]
        return sample

    else:
        indexes = random.sample(range(df_size), sample_size)
        sample = DataFrame.iloc[indexes, : ]
        return sample

In [ ]:
#import data
atlas = pd.read_csv('./Atlases/tissue_predictor_notfiltered_healthy_nofluid.csv', sep="/")
atlas.head()

In [ ]:
print(atlas.shape)

In [ ]:
atlas=atlas[atlas['tissue_name'] != 'Dental plaque']
atlas=atlas[atlas['tissue_name'] != 'hMSC']
atlas=atlas[atlas['tissue_name'] != 'Hela']
atlas=atlas[atlas['tissue_name'] != 'Unknown']
atlas = atlas.replace({'Cervix': 'Uterine cervix'})
print(atlas.shape)

## Check the class balance

In [ ]:
atlas['tissue_name'].value_counts().plot.pie()

In [ ]:
atlas['tissue_name'].value_counts().plot(kind='bar')

We will balance the atlas a little bit by dropping the tissues that are too low. 

In [ ]:
pd.set_option('display.max_rows', None)  
tissue_counts = atlas['tissue_name'].value_counts().to_frame()
print(tissue_counts)

# Drop low tissues

In [ ]:
low_tissues = tissue_counts.index[tissue_counts['tissue_name']<3].tolist()
atlas = atlas[~atlas['tissue_name'].isin(low_tissues)]

In [ ]:
tf = dict(Counter(atlas['tissue_name']))
tf = sorted(tf.items(), key=operator.itemgetter(1), reverse=True)
tf = dict(tf)

In [ ]:

tf

# Using class weight in the predictors <br>
To further balance the algorithm, the class will be assigned a specific weight based on the number of samples in this class. 

The weight of a class is determined by dividing negative samples/ positive samples. So if a class contains 15 samples in a dataset of 100 samples, the class weight will be 85/15=5,667

In [ ]:
atlas=atlas.drop(columns=['cell_type', 'disease_status', 'fluid'])
atlas.head()

In [ ]:
X = atlas.iloc[:, :-1]
y = atlas[['tissue_name']]

In [ ]:
som = atlas.shape[0]
weight_and_label = pd.DataFrame(columns=['label', 'weight'])
i = 0
for key, value in tf.items():
    i += 1
    w = (som - value)/value
    weight_and_label.loc[i] = [key, w]

In [ ]:
train_df = pd.DataFrame()
test_df = pd.DataFrame()
validation_df = pd.DataFrame()

tissues = atlas['tissue_name'].unique()
DataFrameDict = {elem : pd.DataFrame for elem in tissues}
for key in DataFrameDict.keys():
    DataFrameDict[key] = atlas[:][atlas['tissue_name'] == key]

for key in DataFrameDict.keys():
    train = sampleData(DataFrameDict[key], 0.80)
    train_df = train_df.append(train)

    test = DataFrameDict[key].drop(train.index)
    test_df = test_df.append(test)

y_train = train_df.pop('tissue_name').values
X_train = train_df.values
y_test = test_df.pop('tissue_name').values
X_test = test_df.values

X_train = pd.DataFrame(X_train, columns=(atlas.columns)[:-1])
X_test = pd.DataFrame(X_test, columns=(atlas.columns)[:-1])

In [ ]:
train_label_weight = pd.merge(pd.DataFrame(y_train, columns=['label']), weight_and_label, how='left', on='label')
label_weight = pd.merge(pd.DataFrame(atlas['tissue_name'], columns=['label']), weight_and_label, how='left', on='label')

The weights are linked to the targets in the training data and in the complete dataset.

In [ ]:
train_weights = train_label_weight['weight'].to_numpy()
train_weights = train_weights.flatten()
train_weights

weights = label_weight['weight'].to_numpy()
weights = weights.flatten()
weights

In [ ]:
dict_train_label_weight = train_label_weight.drop_duplicates()
dict_train_label_weights = dict(zip(train_label_weight.label, train_label_weight.weight))

# Comparing predictive algortihms performance on the training data <br>
Using the cross val scores, several metrics of several algorithms are gathered to compare performance and choose 1 algorithm for further tuning.

Each model had the necessary parameters to handle multilabel classification eg. the multisoftprob objective in XGBoost

In [ ]:
# Define a function that compares the CV perfromance of a set of predetrmined models 
#https://towardsdatascience.com/cross-validation-and-hyperparameter-tuning-how-to-optimise-your-machine-learning-model-13f005af9d7d
def cv_comparison(models, names, X, y, cv):
    # Initiate a DataFrame for the averages and a list for all measures
    cv_scores = pd.DataFrame()
    accs = []
    f1s = []
    precs = []
    recs = []
    f1s_w = []
    precs_w = []
    recs_w = []

    # Loop through the models, run a CV, add the average scores to the DataFrame and the scores of 
    # all CVs to the list
    for model, name in zip(models, names):
        print(name)
        start = time.time()
        acc = np.round(cross_val_score(model, X, y, scoring='accuracy', cv=cv), 4)
        accs.append(acc)
        acc_avg = round(np.mean(acc[~np.isnan(acc)]), 4)
        f1 = np.round(cross_val_score(model, X, y, scoring='f1_macro', cv=cv), 4)
        f1s.append(f1)
        f1_avg = round(np.mean(f1[~np.isnan(f1)]), 4)
        prec = np.round(cross_val_score(model, X, y, scoring='precision_macro', cv=cv), 4)
        precs.append(prec)        
        prec_avg = round(np.mean(prec[~np.isnan(prec)]), 4)
        rec = np.round(cross_val_score(model, X, y, scoring='recall_macro', cv=cv), 4)
        recs.append(rec)        
        rec_avg = round(np.mean(rec[~np.isnan(rec)]), 4)

        f1_w = np.round(cross_val_score(model, X, y, scoring='f1_weighted', cv=cv), 4)
        f1s_w.append(f1_w)
        f1_w_avg = round(np.mean(f1_w[~np.isnan(f1_w)]), 4)
        prec_w = np.round(cross_val_score(model, X, y, scoring='precision_weighted', cv=cv), 4)
        precs_w.append(prec_w)        
        prec_w_avg = round(np.mean(prec_w[~np.isnan(prec_w)]), 4)
        rec_w = np.round(cross_val_score(model, X, y, scoring='recall_weighted', cv=cv), 4)
        recs_w.append(rec_w)        
        rec_w_avg = round(np.mean(rec_w[~np.isnan(rec_w)]), 4)
        cv_scores[str(name)] = [acc_avg, f1_avg, prec_avg, rec_avg, f1_w_avg, prec_w_avg, rec_w_avg]
        print(time.time() - start)
    cv_scores.index = ['Accuracy', 'f1_macro', 'precision_macro', 'recall_macro', 'f1_weighted', 'precision_weighted', 'recall_weighted']
    return cv_scores,accs, f1s, precs, recs, f1s_w, precs_w, recs_w

In [ ]:
num_classes =len(np.unique(y_train))
print(num_classes)

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold

log_unbal = LogisticRegression(random_state=42, multi_class='multinomial', n_jobs=-1)
log=LogisticRegression(random_state=42, multi_class='multinomial', class_weight=dict_train_label_weights, n_jobs=-1)
log_bal=LogisticRegression(random_state=42, multi_class='multinomial', class_weight='balanced', n_jobs=-1)

rf_unbal = RandomForestClassifier(random_state=42, n_jobs=-1)
rf=RandomForestClassifier(random_state=42, class_weight=dict_train_label_weights, n_jobs=-1)
rf_bal=RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1)
rf_sub=RandomForestClassifier(random_state=42, class_weight='balanced_subsample', n_jobs=-1)

brf_unbal = BalancedRandomForestClassifier(random_state=42, n_jobs=-1)
brf = BalancedRandomForestClassifier(random_state=42, class_weight=dict_train_label_weights, n_jobs=-1)
brf_bal = BalancedRandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1)
brf_sub = BalancedRandomForestClassifier(random_state=42, class_weight='balanced_subsample', n_jobs=-1)

xgb_unbal = XGBClassifier(random_state=42,objective='multi:softprob', eval_metric='mlogloss', num_class=num_classes, n_jobs=-1)
xgb=XGBClassifier(random_state=42, objective='multi:softprob', eval_metric='mlogloss', num_class=num_classes, weight=train_weights, n_jobs=-1)

svm_unbal=SVC(random_state=42)
svm=SVC(random_state=42, class_weight=dict_train_label_weights)

In [ ]:
models=[log_unbal, log, log_bal,rf, rf_bal, rf_sub, brf_unbal, brf, brf_bal, brf_sub,xgb_unbal, xgb,svm_unbal, svm]
names = ['LogisticRegression unbalanced', 'LogisticRegression dict balanced', 'LogisticRegression balanced','RandomForest unbalanced', 'RandomForest dict balanced', 'RandomForest balanced', 'Random Forest balanced subsample', 'Balanced RandomForest',
'Balanced RandomForest balanced','Balanced RandomForest balanced subsample','XGBClassifier unbalanced', 'XGBClassifier dict balanced','SVM unbalanced', 'SVM']
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)
comp1, accs, f1s, precs, recs, f1s_w, precs_w, recs_w = cv_comparison(models, names, X_train, y_train, cv=cv)

# Visualisation of performance

In [ ]:
import pandas as pd
comp=pd.read_csv('comparison_full_data.csv', sep='\t')
comp = comp.set_index('Metrics')
comp.head()

In [ ]:
ct = comp.transpose()

In [ ]:
fig, ax = plt.subplots(figsize=(14,8))
ax = sns.heatmap(ct, annot=True, cmap="magma", fmt=".3")
ax.xaxis.tick_top()
plt.show()

# RandomForest

In [ ]:
result_df = pd.DataFrame(columns=['model','Accuracy', 'f1_macro', 'precision_macro', 'recall_macro', 'f1_weighted', 'precision_weighted', 'recall_weighted'])
result_df_cv = pd.DataFrame(columns=['model','fold', 'Accuracy', 'f1_macro', 'precision_macro', 'recall_macro', 'f1_weighted', 'precision_weighted', 'recall_weighted'])

In [ ]:
forest= RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1)
forest.fit(X_train, y_train)
baseline_predictions =  forest.predict(X_test)
fm = f1_score(y_test, baseline_predictions, average="macro")
fw = f1_score(y_test, baseline_predictions, average="weighted")
acc = accuracy_score(y_test, baseline_predictions)
pw = precision_score(y_test, baseline_predictions, average='weighted')
pm = precision_score(y_test, baseline_predictions, average='macro')
rw = recall_score(y_test, baseline_predictions, average='weighted')
rm = recall_score(y_test, baseline_predictions, average='macro')
df_length = len(result_df)
result_df.loc[df_length] = ['RandomForest_baseline', acc, fm, pm, rm, fw, pm, rw]

In [ ]:
result_df

In [ ]:
fold_number = 0
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
for train_index, test_index in skf.split(X, y):
    fold_number += 1
    train = atlas.iloc[train_index,:]
    X_train_cv = train.iloc[:,:-1]
    y_train_cv = train.iloc[:,-1]
    test = atlas.iloc[test_index,:]
    X_test_cv = test.iloc[:, :-1]
    y_test_cv = test.iloc[:,-1]
    forest = RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1)
    forest.fit(X_train_cv, y_train_cv)
    importances = forest.feature_importances_
    baseline_predictions =  forest.predict(X_test_cv)
    fm = f1_score(y_test_cv, baseline_predictions, average="macro")
    fw = f1_score(y_test_cv, baseline_predictions, average="weighted")
    acc = accuracy_score(y_test_cv, baseline_predictions)
    pw = precision_score(y_test_cv, baseline_predictions, average='weighted')
    pm = precision_score(y_test_cv, baseline_predictions, average='macro')
    rw = recall_score(y_test_cv, baseline_predictions, average='weighted')
    rm = recall_score(y_test_cv, baseline_predictions, average='macro')
    df_length = len(result_df_cv)
    result_df_cv.loc[df_length] = ['RandomForest_baseline',fold_number, acc, fm, pm, rm, fw, pm, rw]
print(result_df_cv.mean())

In [ ]:
result_df_cv

In [ ]:
n_estimators = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000]
max_features = ['log2', 'sqrt']
max_depth = [1, 2, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110]
min_samples_split = [1, 2, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
min_samples_leaf = [1, 2, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
bootstrap = [True, False]
param_dist = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rs = RandomizedSearchCV(forest,#sample weights are already in the forest algorithm so no need to add again
param_dist,
n_iter=100,
cv=3,
verbose=2,
random_state=0)

rs.fit(X_train, y_train)
rs.best_params_

In [ ]:
rs_df = pd.DataFrame(rs.cv_results_).sort_values('rank_test_score').reset_index(drop=True)
rs_df = rs_df.drop([
            'mean_fit_time', 
            'std_fit_time', 
            'mean_score_time',
            'std_score_time', 
            'params', 
            'split0_test_score', 
            'split1_test_score', 
            'split2_test_score', 
            'std_test_score'],
            axis=1)
rs_df.head(10)

In [ ]:
n_estimators = [650,500,50,400,200]
max_features = ['sqrt', 'log2']
max_depth = [70,90,30,50]
min_samples_split = [25,2,20,30]
min_samples_leaf = [1,2,10]
bootstrap = [True, False]
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

gs = GridSearchCV(forest,
param_grid,
cv=3,
verbose=2)

gs.fit(X_train, y_train)
gs.best_params_

In [ ]:
forest_opt=RandomForestClassifier(n_estimators=650, max_depth=90, max_features='sqrt', bootstrap=False,
    min_samples_leaf=1, min_samples_split=20, random_state=42, class_weight='balanced', n_jobs=-1)
forest_opt.fit(X_train, y_train)
baseline_predictions =  forest_opt.predict(X_test)
fm = f1_score(y_test, baseline_predictions, average="macro")
fw = f1_score(y_test, baseline_predictions, average="weighted")
acc = accuracy_score(y_test, baseline_predictions)
pw = precision_score(y_test, baseline_predictions, average='weighted')
pm = precision_score(y_test, baseline_predictions, average='macro')
rw = recall_score(y_test, baseline_predictions, average='weighted')
rm = recall_score(y_test, baseline_predictions, average='macro')
df_length = len(result_df)
result_df.loc[df_length] = ['RandomForest_optimised', acc, fm, pm, rm, fw, pm, rw]

In [ ]:
fold_number = 0
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
for train_index, test_index in skf.split(X, y):
    fold_number += 1
    train = atlas.iloc[train_index,:]
    X_train_cv = train.iloc[:,:-1]
    y_train_cv = train.iloc[:,-1]
    test = atlas.iloc[test_index,:]
    X_test_cv = test.iloc[:, :-1]
    y_test_cv = test.iloc[:,-1]
    forest_optcv = RandomForestClassifier(n_estimators=650, max_depth=90, max_features='sqrt', bootstrap=False,
    min_samples_leaf=1, min_samples_split=20, random_state=42, class_weight='balanced', n_jobs=-1)
    forest_optcv.fit(X_train_cv, y_train_cv)
    optimized_predictionscv =  forest_optcv.predict(X_test_cv)
    fm = f1_score(y_test_cv, optimized_predictionscv, average="macro")
    fw = f1_score(y_test_cv, optimized_predictionscv, average="weighted")
    acc = accuracy_score(y_test_cv, optimized_predictionscv)
    pw = precision_score(y_test_cv, optimized_predictionscv, average='weighted')
    pm = precision_score(y_test_cv, optimized_predictionscv, average='macro')
    rw = recall_score(y_test_cv, optimized_predictionscv, average='weighted')
    rm = recall_score(y_test_cv, optimized_predictionscv, average='macro')
    df_length = len(result_df_cv)
    result_df_cv.loc[df_length] = ['RandomForest_optimised',fold_number, acc, fm, pm, rm, fw, pm, rw]
print(result_df_cv.mean())

In [ ]:
result_df

In [ ]:
result_df_cv

In [ ]:
def plot_confusion_matrix(cm, y_test, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(np.unique(y_test)))
    plt.xticks(tick_marks, np.unique(y_test), rotation=90)
    plt.yticks(tick_marks, np.unique(y_test))
    plt.tight_layout()
    plt.grid(color='gainsboro')
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
from sklearn.metrics import confusion_matrix

# Compute confusion matrix
cm = confusion_matrix(y_test, baseline_predictions)
np.set_printoptions(precision=2)
print('Confusion matrix, without normalization')
print(cm)
plt.figure(figsize=(20,10))
plot_confusion_matrix(cm, y_test)

# Normalize the confusion matrix by row (i.e by the number of samples
# in each class)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Normalized confusion matrix')
print(cm_normalized)
plt.figure(figsize=(20,12), dpi=1200)
plot_confusion_matrix(cm_normalized, y_test, title='Normalized confusion matrix')

In [ ]:
filename = 'tissue_predictor_RF_full_opt.pkl'
pickle.dump(forest, open(filename, 'wb'))